# Problem 2 - Neural Network Training and Backpropagation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline
#Load data and create X and y variables
data = loadmat('/content/sample_data/ex3data1.mat')
X = data['X']
y = data['y']
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [ ]:
X.shape, y.shape

((5000, 400), (5000, 1))

In [ ]:
#One-hot encode y labels to transform 'y' variables into 0 and 1 values
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
print(y_onehot.shape)
y[0], y_onehot[0,:]

(5000, 10)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(array([10], dtype=uint8), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]))

## 2.1

In [ ]:
#Sigmoid Function
def sigmoid(z):
  #Scaling the input 'z' for the sigmoid function makes the curve steeper
  return 1 / (1 + np.exp(-2 * z))

## 2.2

Parameters:

    - X → Input features
    - theta1, theta2, theta3 → Weights for each layer
Returns:

    - Activations and weighted sums for each layer


---



In [ ]:
#Forward_propagate() Function
def forward_propagate(X, theta1, theta2, theta3):
  m = X.shape[0]
  #Layer 1: Input Layer
  a1 = np.insert(X, 0, values=np.ones(m), axis=1)
  #Layer 2: First Hidden Layer
  z2 = a1 * theta1.T #Changed from element-wise multiplication to matrix multiplication
  a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1) #Using the scaled sigmoid function
  #Layer 3: Second Hidden Layer
  z3 = a2 * theta2.T
  a3 = np.insert(sigmoid(z3), 0, values=np.ones(m), axis=1)
  #Layer 4: Output Layer
  z4 = a3 * theta3.T
  h = sigmoid(z4)
  return a1, z2, a2, z3, a3, z4, h

## 2.3

In [ ]:
#Cost() Function Without Regularization
def cost_no_regularization(params, input_size, hidden_size1, hidden_size2, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # Reshape params to get weights for each layer
    limit0 = hidden_size1 * (input_size + 1)
    limit1 = limit0 + hidden_size2 * (hidden_size1 + 1)
    theta1 = np.reshape(params[:limit0], (hidden_size1, input_size + 1))
    theta2 = np.reshape(params[limit0:limit1], (hidden_size2, hidden_size1 + 1))
    theta3 = np.reshape(params[limit1:], (num_labels, hidden_size2 + 1))

    # Forward propagate
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    # Compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    return J / m

In [ ]:
#Cost() Function With Regularization
def cost_with_regularization(params, input_size, hidden_size1, hidden_size2, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # Reshape params to get weights for each layer
    limit0 = hidden_size1 * (input_size + 1)
    limit1 = limit0 + hidden_size2 * (hidden_size1 + 1)
    theta1 = np.reshape(params[:limit0], (hidden_size1, input_size + 1))
    theta2 = np.reshape(params[limit0:limit1], (hidden_size2, hidden_size1 + 1))
    theta3 = np.reshape(params[limit1:], (num_labels, hidden_size2 + 1))

    # Forward propagate
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    # Compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    # Add regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) +
                                             np.sum(np.power(theta2[:,1:], 2)) +
                                             np.sum(np.power(theta3[:,1:], 2)))

    return J

In [ ]:
# initial setup
input_size = 400
hidden_size1 = 20
hidden_size2 = 20
num_labels = 10
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size1 * (input_size + 1) + hidden_size2 * (hidden_size1 + 1) + num_labels * (hidden_size2 + 1)) - 0.5) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

theta1 = np.matrix(np.reshape(params[:hidden_size1 * (input_size + 1)], (hidden_size1, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size1 * (input_size + 1):hidden_size1 * (input_size + 1) + hidden_size2 * (hidden_size1 + 1)], (hidden_size2, (hidden_size1 + 1))))
theta3 = np.matrix(np.reshape(params[hidden_size1 * (input_size + 1) + hidden_size2 * (hidden_size1 + 1):], (num_labels, (hidden_size2 + 1))))
theta1.shape, theta2.shape, theta3.shape

((20, 401), (20, 21), (10, 21))

In [ ]:
a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)
a1.shape, z2.shape, a2.shape, z3.shape, a3.shape, z4.shape, h.shape

((5000, 401),
 (5000, 20),
 (5000, 21),
 (5000, 20),
 (5000, 21),
 (5000, 10),
 (5000, 10))

In [ ]:
cost_no_regularization(params, input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot, learning_rate)

6.902314128126856

In [ ]:
cost_with_regularization(params, input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot, learning_rate)

6.906787101983319

## 2.4

In [ ]:
#Sigmoid_gradient() Function
def sigmoid_gradient(z):
    return 2 * np.multiply(sigmoid(z), (1 - sigmoid(z)))

## 2.5

In [ ]:
#Backprop() Without Regularization
def backprop(params, input_size, hidden_size1, hidden_size2, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size1 * (input_size + 1)], (hidden_size1, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size1 * (input_size + 1):hidden_size1 * (input_size + 1) + hidden_size2 * (hidden_size1 + 1)], (hidden_size2, (hidden_size1 + 1))))
    theta3 = np.matrix(np.reshape(params[hidden_size1 * (input_size + 1) + hidden_size2 * (hidden_size1 + 1):], (num_labels, (hidden_size2 + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    delta3 = np.zeros(theta3.shape)

    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))

    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]
        z2t = z2[t,:]
        a2t = a2[t,:]
        z3t = z3[t,:]
        a3t = a3[t,:]
        ht = h[t,:]
        yt = y[t,:]

        d4t = ht - yt
        z3t = np.insert(z3t, 0, values=np.ones(1))
        d3t = np.multiply((theta3.T * d4t.T).T, sigmoid_gradient(z3t))

        z2t = np.insert(z2t, 0, values=np.ones(1))
        d2t = np.multiply((theta2.T * d3t[:,1:].T).T, sigmoid_gradient(z2t))

        delta1 += (d2t[:,1:]).T * a1t
        delta2 += (d3t[:,1:]).T * a2t
        delta3 += d4t.T * a3t

    delta1 = delta1 / m
    delta2 = delta2 / m
    delta3 = delta3 / m

    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2), np.ravel(delta3)))

    return J, grad

In [ ]:
J, grad = backprop(params, input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot, learning_rate)
J, grad.shape

(6.906695011130548, (8650,))

In [ ]:
#Backprop() With Regularization
def backprop_with_regularization(params, input_size, hidden_size1, hidden_size2, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    #Reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size1 * (input_size + 1)], (hidden_size1, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size1 * (input_size + 1):hidden_size1 * (input_size + 1) + hidden_size2 * (hidden_size1 + 1)], (hidden_size2, (hidden_size1 + 1))))
    theta3 = np.matrix(np.reshape(params[hidden_size1 * (input_size + 1) + hidden_size2 * (hidden_size1 + 1):], (num_labels, (hidden_size2 + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    #Initializations
    J = 0
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    delta3 = np.zeros(theta3.shape)

    #Compute the cost with regularization
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)) + np.sum(np.power(theta3[:,1:], 2)))

    #Perform backpropagation with regularization
    for t in range(m):
        a1t = a1[t,:]
        z2t = z2[t,:]
        a2t = a2[t,:]
        z3t = z3[t,:]
        a3t = a3[t,:]
        ht = h[t,:]
        yt = y[t,:]

        d4t = ht - yt
        z3t = np.insert(z3t, 0, values=np.ones(1))
        d3t = np.multiply((theta3.T * d4t.T).T, sigmoid_gradient(z3t))

        z2t = np.insert(z2t, 0, values=np.ones(1))
        d2t = np.multiply((theta2.T * d3t[:,1:].T).T, sigmoid_gradient(z2t))

        delta1 += (d2t[:,1:]).T * a1t
        delta2 += (d3t[:,1:]).T * a2t
        delta3 += d4t.T * a3t

    delta1 = delta1 / m
    delta2 = delta2 / m
    delta3 = delta3 / m

    delta1[:,1:] += (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] += (theta2[:,1:] * learning_rate) / m
    delta3[:,1:] += (theta3[:,1:] * learning_rate) / m

    #Unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2), np.ravel(delta3)))

    return J, grad

In [ ]:
J, grad = backprop(params, input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot, learning_rate)
J, grad.shape

(6.906695011130548, (8650,))

## 2.6

In [ ]:
from scipy.optimize import minimize

# Hyperparameters and initial setup
input_size = 400   #Each input feature vector is of size 400
hidden_size1 = 20  #Number of units in the first hidden layer
hidden_size2 = 20  #Number of units in the second hidden layer
num_labels = 10    #Number of output classes
learning_rate = 1  #As mentioned in the question to keep unchanged

# Randomly initialize weights
params = (np.random.random(size=hidden_size1 * (input_size + 1) + hidden_size2 * (hidden_size1 + 1) + num_labels * (hidden_size2 + 1)) - 0.5) * 0.25

# Minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot, learning_rate),
                method='TNC', jac=True, options={'maxiter': 250})

# Extract trained parameters
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size1 * (input_size + 1)], (hidden_size1, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size1 * (input_size + 1):hidden_size1 * (input_size + 1) + hidden_size2 * (hidden_size1 + 1)], (hidden_size2, (hidden_size1 + 1))))
theta3 = np.matrix(np.reshape(fmin.x[hidden_size1 * (input_size + 1) + hidden_size2 * (hidden_size1 + 1):], (num_labels, (hidden_size2 + 1))))

<ipython-input-126-c3104bb7c335>:14: OptimizeWarning: Unknown solver options: maxiter
  fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot, learning_rate),


## 2.7

In [ ]:
a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred

array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]])

In [ ]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print('accuracy = {0}%'.format(accuracy * 100))

accuracy = 99.68%


## 2.8

**Answer:**

- The 3-layer neural network model that we implemented showed a slight improvement in accuracy, achieving 99.68%, compared to the 2-layer neural network from the original notebook which had an accuracy of 99.22%.
- This suggests that the additional hidden layer in our model may have captured more complex features or patterns in the data, leading to a marginally better performance.
- However, it is essential to note that while the improvement is positive, it's relatively minor. In reality, the decision to add more layers should be balanced with the potential risk of overfitting.
- In this case, the addition of an extra layer did provide a performance boost, albeit a small one.